In [1]:
!pip install webdriver_manager
!pip install splinter

## Step 1 - Scraping

In [2]:
# Dependencies
import os
from bs4 import BeautifulSoup
import requests
import pymongo
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

### NASA Mars News

In [3]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240


[WDM] - Driver [/Users/jessicafernandez/.wdm/drivers/chromedriver/mac64/86.0.4240.22/chromedriver] found in cache


In [4]:
# URL of page to be scraped
url_news = 'https://mars.nasa.gov/news/'

# Retrieve page with the requests module
response = requests.get(url_news)
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
# pull titles and body from website
results = soup.find_all('div', class_="slide")
for result in results:
    titles = result.find('div', class_="content_title")
    title = titles.find('a').text
    bodies = result.find('div', class_="rollover_description")
    body = bodies.find('div', class_="rollover_description_inner").text
print('----------------')
print(title)
print(body)

----------------

NASA's Perseverance Mars Rover Gets Its Wheels and Air Brakes


After the rover was shipped from JPL to Kennedy Space Center, the team is getting closer to finalizing the spacecraft for launch later this summer.



### JPL Mars Space Images - Featured Image

In [6]:
# URL of page to be scraped
url_img = ('https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars')

# Retrieve page with the requests module
response = requests.get(url_img)

# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

In [7]:
# pull images from website
images = soup.find_all('a', class_="fancybox")
print(images)

[<a class="button fancybox" data-description="Each dot represents an asteroid or comet that the mission observed." data-fancybox-group="images" data-fancybox-href="/spaceimages/images/mediumsize/PIA19101_ip.jpg" data-link="/spaceimages/details.php?id=PIA19101" data-title="One Year of NEOWISE Observations Mapped" id="full_image">
					FULL IMAGE
				  </a>, <a class="fancybox" data-description="This image from NASAs Mars Odyssey shows linear depressions, part of Sirenum Fossae." data-fancybox-group="images" data-fancybox-href="/spaceimages/images/largesize/PIA24244_hires.jpg" data-link="/spaceimages/details.php?id=PIA24244" data-thumbnail="/spaceimages/images/wallpaper/PIA24244-640x350.jpg" data-title="Sirenum Fossae">
<div class="image_and_description_container">
<div class="rollover_description">
<h3 class="release_date">November 12, 2020</h3>
<div class="item_tease_overlay">Sirenum Fossae</div>
<div class="overlay_arrow">
<img alt="more arrow" src="/assets/images/overlay-arrow.png"/>

In [8]:
# pull image link
src = []
for image in images:
    pic = image['data-fancybox-href']
    src.append(pic)

featured_image_url = 'https://www.jpl.nasa.gov' + pic
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA24144_hires.jpg'

### Mars Facts

In [10]:
mars_facts_url = "https://space-facts.com/mars/"
table = pd.read_html(mars_facts_url)
table

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [11]:
df=table[0]
df.columns = ["Description", "Data"]
df.set_index(["Description"])

,Data
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [12]:
html_table = df.to_html(index=False)
html_table.replace('\n','')
df.to_html('table.html')

In [13]:
!open table.html

### Mars Hemispheres

In [14]:
url_hem = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(url_hem)

import time 
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
mars_hemisphere=[]

In [15]:
# loop through the four tags and load the data to the dictionary

for i in range (4):
    time.sleep(5)
    images = browser.find_by_tag('h3')
    images[i].click()
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    partial = soup.find("img", class_="wide-image")["src"]
    img_title = soup.find("h2",class_="title").text
    img_url = 'https://astrogeology.usgs.gov'+ partial
    dictionary={"title":img_title,"img_url":img_url}
    mars_hemisphere.append(dictionary)
    browser.back()

In [16]:
print(mars_hemisphere)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]


## Step 2 - MongoDB and Flask Application